In [ ]:
import cv2
import numpy as np
import tensorflow as tf
from collections import deque
import os

# === CONFIG ===
keras_model_path = "Keras/C1gesture_Epoch90.keras"
tflite_model_path = "Keras/C1gesture_Epoch90.keras.keras.tflite"
frame_window = 10
image_size = (128, 128)
angle_labels = [f"angle_{i}" for i in range(1, 7)]
no_wave_label = "noWave"
all_labels = angle_labels + [no_wave_label]

# === Step 1: Convert to INT8 TFLite (if not already exists) ===
if not os.path.exists(tflite_model_path):
    print("🔁 Converting Keras model to INT8 TFLite...")

    # Load Keras model
    model = tf.keras.models.load_model(keras_model_path)

    # Define representative dataset
    def representative_data_gen():
        for _ in range(100):
            dummy_img = np.random.rand(*image_size, 3).astype(np.float32)
            dummy_img = np.expand_dims(dummy_img, axis=0)
            yield [dummy_img]

    # Convert to int8 TFLite
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    converter.optimizations = [tf.lite.Optimize.DEFAULT]
    converter.representative_dataset = representative_data_gen
    converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
    converter.inference_input_type = tf.uint8
    converter.inference_output_type = tf.uint8

    tflite_model = converter.convert()
    with open(tflite_model_path, "wb") as f:
        f.write(tflite_model)

    print(f"✅ Saved quantized model to: {tflite_model_path}")
else:
    print(f"📦 Found existing quantized model: {tflite_model_path}")

# === Step 2: Load TFLite model ===
interpreter = tf.lite.Interpreter(model_path=tflite_model_path)
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# === Step 3: Set up camera and prediction history ===
pred_history = deque(maxlen=frame_window)
cap = cv2.VideoCapture(0)
i = 0

print("🚀 INT8 Handwave detector running... Press Q to quit.")

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # === Preprocess frame ===
    resized = cv2.resize(frame, image_size)
    input_scale, input_zero_point = input_details[0]['quantization']
    input_tensor = resized.astype(np.float32) / 255.0
    input_tensor = input_tensor / input_scale + input_zero_point
    input_tensor = np.clip(input_tensor, 0, 255).astype(np.uint8)
    input_tensor = np.expand_dims(input_tensor, axis=0)

    # === TFLite Inference ===
    interpreter.set_tensor(input_details[0]['index'], input_tensor)
    interpreter.invoke()
    output = interpreter.get_tensor(output_details[0]['index'])

    pred_label = all_labels[np.argmax(output)]
    pred_history.append(pred_label)

    # === Display prediction on video ===
    cv2.putText(frame, f"Prediction: {pred_label}", (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.imshow('Live Handwave Detector (INT8 TFLite)', frame)

    # === Detect handwave ===
    unique_angles = set(label for label in pred_history if label in angle_labels)
    if len(unique_angles) >= 3:
        i += 1
        print(f"🖐️ Handwave Detected! (Detected angles: {sorted(unique_angles)}) → Hello World! ", i)
        pred_history.clear()

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


🔁 Converting Keras model to INT8 TFLite...
INFO:tensorflow:Assets written to: C:\Users\josep\AppData\Local\Temp\tmpd19s6dmb\assets


INFO:tensorflow:Assets written to: C:\Users\josep\AppData\Local\Temp\tmpd19s6dmb\assets


Saved artifact at 'C:\Users\josep\AppData\Local\Temp\tmpd19s6dmb'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 128, 128, 3), dtype=tf.float32, name='input_layer')
Output Type:
  TensorSpec(shape=(None, 7), dtype=tf.float32, name=None)
Captures:
  2162780670608: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2162780676592: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2162780683984: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2162780850128: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2162780679408: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2162780681696: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2162780854880: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2162780852944: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2162780850832: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2162780857344: TensorSpec(shape=(), dtype=tf.resource, name=None)
  216278

C:\Users\josep\anaconda3\envs\ai_projects-env\lib\site-packages\tensorflow\lite\python\convert.py:854: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(


✅ Saved quantized model to: Keras/C1gesture_Epoch90.keras.keras.tflite


C:\Users\josep\anaconda3\envs\ai_projects-env\lib\site-packages\tensorflow\lite\python\interpreter.py:457: UserWarning:     Warning: tf.lite.Interpreter is deprecated and is scheduled for deletion in
    TF 2.20. Please use the LiteRT interpreter from the ai_edge_litert package.
    See the [migration guide](https://ai.google.dev/edge/litert/migration)
    for details.
    
  warnings.warn(_INTERPRETER_DELETION_WARNING)


🚀 INT8 Handwave detector running... Press Q to quit.
🖐️ Handwave Detected! (Detected angles: ['angle_1', 'angle_4', 'angle_6']) → Hello World!  1
🖐️ Handwave Detected! (Detected angles: ['angle_1', 'angle_4', 'angle_6']) → Hello World!  2
🖐️ Handwave Detected! (Detected angles: ['angle_1', 'angle_4', 'angle_6']) → Hello World!  3
🖐️ Handwave Detected! (Detected angles: ['angle_1', 'angle_4', 'angle_6']) → Hello World!  4
🖐️ Handwave Detected! (Detected angles: ['angle_1', 'angle_4', 'angle_6']) → Hello World!  5
🖐️ Handwave Detected! (Detected angles: ['angle_1', 'angle_4', 'angle_6']) → Hello World!  6
🖐️ Handwave Detected! (Detected angles: ['angle_1', 'angle_4', 'angle_6']) → Hello World!  7
🖐️ Handwave Detected! (Detected angles: ['angle_1', 'angle_4', 'angle_6']) → Hello World!  8
🖐️ Handwave Detected! (Detected angles: ['angle_1', 'angle_4', 'angle_6']) → Hello World!  9
🖐️ Handwave Detected! (Detected angles: ['angle_1', 'angle_4', 'angle_6']) → Hello World!  10
🖐️ Handwave Dete